8 Théorie des graphes


Un graphe est un couple $G = (X, E)$ constitué d'un ensemble $X$, non vide et fini, et d'un ensemble $E$ de paires d'éléments de $X$. Les éléments de $X$ sont les sommets du graphe $G$, ceux de $E$ sont les arêtes du graphe $G$.
Un graphe est orienté si les arêtes ont une direction, c'est-à-dire si les couples d'éléments de $E$ sont des listes ordonnées telles que $(i,j) \in E$ n'est pas équivalent à $(j,i)\in E$. Ici seuls des graphes non orientés, c'est-à-dire dont les paires d'éléments de $X$ sont des ensembles non ordonnés ($\{i,j\} \in E$), sont considérés.

Par exemple, le graphe complet à $n$ sommets $K_n$ est le graphe de sommets $X = {1,2,\dots,n}$ ayant comme arêtes les parties à deux éléments de $X$. En particulier, $K_4 = (X,E)$ où $X=\{1,2,3,4\}$ et $E = \big\{ \{1,2\}, \{1,3\}, \{1,4\}, \{2, 3\}, \{2, 4\}, \{3, 4\}  \big\}$.

**Concepts abordés:**

* graphes non orientés

* représentation comme dictionnaire

* utilisation des frozensets

* matrice d'adjacence

* recherche de chemins et de triangles

* fonction récursive

1. Graphes comme dictionnaires



Une façon de représenter un graphe $G$ est de le faire avec un dictionnaire dont les clefs sont
les sommets, et la valeur associée à chaque clef $x \in X$ est un ensemble contenant les voisins de $x$.


a)
Construisons sous forme de dictionnaire les graphes suivants:

<center><img src="https://python.guillod.org/fig/graphes.png" style="width:90%;max-width:800px;"></center>


In [1]:
import numpy as np

In [6]:
graphA= {1: {2, 5}, 2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}, 6: {4}}
K5= {1: {2, 3, 4, 5}, 2: {1, 3, 4, 5}, 3: {1, 2, 4, 5}, 4: {1, 2, 3, 5}, 5: {1, 2, 3, 4}}
peterson= {1: {2, 5, 6}, 2: {1, 3, 7}, 3: {2, 4, 8}, 4: {3, 5, 9}, 5: {1, 4, 10}, 6: {1, 8, 9}, 7: {2, 9, 10},
           8: {3, 6, 10}, 9: {4, 6, 7}, 10: {5, 7, 8}}

b)
Écrivons une fonction `complet(n)` qui construit comme dictionnaire le graphe complet $K_n$.

In [7]:
def complet(n):
    graph= {i: {j for j in range(1, n+1)} for i in range(1, n+1)}
    for i in graph: 
        graph[i].remove(i)
    return graph
complet(4)

{1: {2, 3, 4}, 2: {1, 3, 4}, 3: {1, 2, 4}, 4: {1, 2, 3}}

c)
Un graphe donné sous forme de dictionnaire contient l'information plusieurs fois. Écrivons une fonction `corriger(graphe)` permettant de rajouter les éléments manquants de sorte que pour tout sommet `x`, si `y` appartient à `graphe[x]`, alors `y` est aussi une clef et `x` appartient à `graphe[y]`. Puis testons cette fonction.

In [8]:
def corriger(graphe):
    for x in graphe.copy():
        for y in graphe[x]:
            if (y not in graphe):
                graphe[y]=set()
            graphe[y].add(x)
graphA= {1: {2, 5}, 2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}, 6: {4}}
del graphA[6]
del graphA[1]
print(graphA)
corriger(graphA)
print(graphA)

{2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}}
{2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}, 1: {2, 5}, 6: {4}}


d)
Écrivons une fonction qui retourne l'ensemble (type `set`) de toutes les arêtes d'un graphe représenté par un dictionnaire.

Note : 
Les ensembles sont mutables et donc pas hashables.

In [9]:
def aretes(graphe):
    res=set()
    for x in graphe:
        for y in graphe[x]:
            if {x,y} not in res:
                res.add(frozenset({x, y}))
    return res
aretes(graphA)

{frozenset({3, 4}),
 frozenset({4, 6}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({4, 5}),
 frozenset({2, 5}),
 frozenset({1, 5})}

e)Écrivons une fonction permettant de déterminer si deux sommets sont connectés par un chemin ou non et qui retourne le chemin si oui.

Note : 
C'est une fonction récursive. Dans l'algorithme ci-après, la variable file est initialement vide, et se remplie et se déremplie à mesure qu'on explore le graphe. Cul_de_sac sert a se souvenir des noeuds qui ne mène nul part. L'exploration se fait en profondeur

In [7]:
def chemin(graphe, a, b, file, cul_de_sac):
    if a not in file:
        file.append(a)
    if a==b:
        return file
    else:
        for y in graphe[a]:
            if y not in file and y not in cul_de_sac:
                return chemin(graphe, y, b, file, cul_de_sac)
        cul_de_sac.add(a)
        file.remove(a)
        if len(file)==0:
            return False
        return chemin(graphe, file[-1], b, file, cul_de_sac)
graphA= {1: {2, 5}, 2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}, 6: {4}}
print(graphA)
print(chemin(graphA, 1, 6, [], set()))
print(peterson)
print(chemin(peterson, 1, 8, [], set()))

{1: {2, 5}, 2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}, 6: {4}}
[1, 2, 3, 4, 6]
{1: {2, 5, 6}, 2: {1, 3, 7}, 3: {8, 2, 4}, 4: {9, 3, 5}, 5: {1, 10, 4}, 6: {8, 1, 9}, 7: {9, 2, 10}, 8: {10, 3, 6}, 9: {4, 6, 7}, 10: {8, 5, 7}}
[1, 2, 3, 8]


f) Écrivons une fonction qui retourne tous les chemins entre deux sommets (sans les cycles).L'idée est d'enlever la dernière arête de chaque chemin trouvé dans un graphe cloné :

In [8]:
def leschemins(graphe, a, b):
    pool=set()
    clone=graphe.copy()
    essai = chemin(clone, a, b, [], set())
    while essai!= False:
        pool.add(frozenset(essai))
        print(essai)
        clone[essai[-2]].remove(essai[-1])
        clone[essai[-1]].remove(essai[-2])
        essai=chemin(clone, a, b, [], set())
    return pool

graphA= {1: {2, 5}, 2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}, 6: {4}}
print(leschemins(graphA, 1, 5))
graphB= {1: {}, 2: {}, 3: {}}
print(leschemins(graphB, 1, 3))


[1, 2, 3, 4, 5]
[1, 2, 5]
[1, 5]
{frozenset({1, 2, 3, 4, 5}), frozenset({1, 5}), frozenset({1, 2, 5})}
set()


2. Triangles dans un graphe

Un triangle dans un graphe est un ensemble de trois sommets reliés entre eux par trois arêtes. La recherche et l'analyse des triangles dans un graphe sont importantes pour comprendre sa structure.


a)
Mathématiquement, on peut déterminer le nombre de sous-ensembles de cardinal trois que possède un ensemble de sommets $X$. Il s'agit de $ \binom{|X|}{3} $

b)
Écrivons une fonction retournant l'ensemble des triangles d'un graphe.

In [3]:
def triangles(graphe):
    pool= set()
    for x in graphe:
        for y in graphe[x]:
            for z in graphe[y]:
                if x in graphe[z] and {x, y, z} not in pool:
                    pool.add(frozenset({x, y, z}))
    return pool

graphA= {1: {2, 5}, 2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}, 6: {4}}
print(graphA)
print(triangles(graphA))
print(K5)
print(len(triangles(K5)))
                

{1: {2, 5}, 2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}, 6: {4}}
{frozenset({1, 2, 5})}
{1: {2, 3, 4, 5}, 2: {1, 3, 4, 5}, 3: {1, 2, 4, 5}, 4: {1, 2, 3, 5}, 5: {1, 2, 3, 4}}
10


c)À chaque graphe $G=(X,E)$ correspond une unique matrice $A$ symétrique de taille $n \times n$ avec $n=|X|$ définie par: 
$$
A_{ij}=\begin{cases}
1\,, & \text{si}\;\{i,j\}\in E\,,\\ 
0\,, & \text{si}\;\{i,j\}\notin E\,.
\end{cases}
$$

Cette matrice est appelée la matrice d'adjacence du graphe $G$.

Définissons une fonction retournant la matrice d'adjacence d'un graphe.

Note : 
Faire attention que les sommets ne sont pas forcément indexés par des entiers compris entre 0 et $n$ dans le dictionnaire.

In [4]:
def adjacence(graphe):
    n= len(graphe)
    A=np.zeros((n,n))
    index=list(graphe.keys())
    for i in range(n):
        for j in range(i+1, n):
            if index[j] in graphe[index[i]]:
                A[i,j]=1
                A[j,i]=1
    return A

graphA= {1: {2, 5}, 2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}, 6: {4}}
print(graphA)
print(adjacence(graphA))

{1: {2, 5}, 2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}, 6: {4}}
[[0. 1. 0. 0. 1. 0.]
 [1. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0.]
 [0. 0. 1. 0. 1. 1.]
 [1. 1. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


d)
Définissons une fonction ayant pour argument une matrice d'adjacence et retournant le graphe correspondant sous forme d'un dictionnaire.

In [10]:
def graphique(A):
    n= len(A)
    graphe={i:set() for i in range(1,n+1)}
    for i in range(1,n+1):
        for j in range(i+1, n+1):
            if A[i-1, j-1]==1:
                graphe[i].add(j)
    corriger(graphe)
    return graphe

graphA= {1: {2, 5}, 2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}, 6: {4}}
print(graphA)
A= adjacence(graphA)
print(graphique(A))

{1: {2, 5}, 2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}, 6: {4}}
{1: {2, 5}, 2: {1, 3, 5}, 3: {2, 4}, 4: {3, 5, 6}, 5: {1, 2, 4}, 6: {4}}


e)
En utilisant la matrice d'adjacence $A$ et la matrice $B=A^2$, écrivons une fonction retournant l'ensemble des triangles d'un graphe. 
$B_{ij}$ est en fait le nombre de chemin de longueur 2 entre les sommets $s_i$ et $s_j$, c'est à dire que si $B_{ij} >0$ et $A_{ij} = 1$, il y a un triangle impliquant $s_i$ et $s_j$. Reste à déterminer le troisième sommet de ce triangle, c'est un sommet indexé par k tel que $A_{ik}=A_{jk}=1$. 

In [11]:
def find_triangles(A):
    N = A.shape[0]
    B = A @ A
    
    triangles = set()
    for i in range(N):
        for j in range(i + 1, N): 
            if A[i, j] == 1:
                num_k = B[i, j]                
                if num_k > 0:
                    for k in range(j + 1, N): # k > j > i
                        if A[i, k] == 1 and A[j, k] == 1:
                            triangles.add(tuple(sorted((i, j, k))))
                            
    return triangles

A_exemple = np.array([
    [0, 1, 1, 0],
    [1, 0, 1, 0], 
    [1, 1, 0, 1],
    [0, 0, 1, 0]
])

triangles_trouves = find_triangles(A_exemple)

print(f"Matrice A:\n{A_exemple}")
print(f"\nMatrice B=A^2:\n{A_exemple @ A_exemple}")
print(f"\nEnsemble des triangles (i, j, k): {triangles_trouves}")

Matrice A:
[[0 1 1 0]
 [1 0 1 0]
 [1 1 0 1]
 [0 0 1 0]]

Matrice B=A^2:
[[2 1 1 1]
 [1 2 1 1]
 [1 1 3 0]
 [1 1 0 1]]

Ensemble des triangles (i, j, k): {(0, 1, 2)}


f)
En utilisant la matrice d'adjacence $A$, écrivons une fonction calculant le nombre de triangles.Je vais utiliser $A^{3}$ pour déterminer le nombre de chemin de longueur 3 commençant et finissant par chaque sommet. $A^{3}_{ii}$ donne ce nombre en double, car chaque chemin peut être parcouru dans un sens et dans l'autre. En sachant que chaque triangle concerne 3 sommets $s_i$ tel que $A^{3}_{ii}=2$, il suffit de calculer la trace de $A^{3}$ et de diviser par 6.

In [12]:
def nbtriangles(graphe):
    A=adjacence(graphe)
    B=A@A@A
    return np.trace(B)/6
print(nbtriangles(graphA))
print(nbtriangles(K5))

1.0
10.0
